# Approximated Harmonic Centrality

### Libraries import & graph retrieval

In [1]:
import pickle               # to read the Amazon DiGraph
import networkx as nx

graph_path = "../data/processed/amazon_graph.pickle"
with open(graph_path, "rb") as f:
    G = pickle.load(f)

---

### Implementation notes

Here are some considerations about the implementation strategies adopted by the following algorithms:

* Both the CPU and GPU versions are implementations of the HyperBall algorithm from [Boldi and Vigna, 2013]
* Since Harmonic Centrality measures how reachable a node is from other nodes, the algorithm works on the transposed graph (`G.reverse()`)
* Counters are stored in a matrix of registers: on the CPU version, each register is stored as a `uint8` to minimize RAM usage (this is enough, since it has to store the number of trailing zeroes in a 128-p bits hash); on the GPU version, each register needs to be stored as a `int32` to ensure compatibility with the `cuPy`
library
* Both implementations simultaneously update the registers using numpy.maximum.at and cupy.maximum.at, based on the values of source nodes
* The iteration stops when the cariation in the cardinality estimate between two consecutive steps drops below a tolerance threshold or when a high iteration litmit is reached.

---

### CPU computation (numPy matrices and buffer pre-allocation)

In [4]:
import hashlib
import pandas as pd
import numpy as np
import time
import gc

def harmonic_CPU(G, p=10):
    print("CPU setup")

    # Graph reverse
    G_rev = G.reverse()

    # Graph representation through nodes' indices
    nodes = list(G_rev.nodes())
    n_nodes = len(nodes)
    node_to_idx = {node: i for i, node in enumerate(nodes)} # mapping node -> node_index
    edges = np.array([(node_to_idx[u], node_to_idx[v]) for u, v in G_rev.edges()], dtype=np.int32)


    # Counters matrix
    m = 1 << p  # registers per counter, m = 2^p
    M_A = np.zeros((n_nodes, m), dtype=np.uint8)
    """
    [n_nodes x m] counter matrix; each register stores the number of trailing zeros of a 128-bit hash, max stored value < 128.
    """

    # Hash computation
    print("Computing hashes on CPU...")
    for i, node in enumerate(nodes):
        # Node hash
        h = int(hashlib.md5(str(node).encode('utf8')).hexdigest(), 16)

        # Bitwise AND to get the last p bits of h, then right shift to remove them
        j = h & (m - 1)
        w = h >> p

        # Count trailing zeroes on the remaining hash (+1)
        rho = 1
        while (w & 1) == 0 and rho < (128 - p):
            w >>= 1
            rho += 1
        M_A[i, j] = rho

    # Hyperball constants
    alpha_m = 0.7213 / (1 + 1.079 / m)
    factor = alpha_m * (m ** 2)

    # Memory pre-allocation
    M_B = M_A.copy()
    M_float = np.empty((n_nodes, m), dtype=np.float32)
    sources = edges[:, 1] # Column 1 of edges
    targets = edges[:, 0] # Column 0 of edges
    # Arrays to store the cardinality at steps t-1 and t
    prev_cardinality = np.ones(n_nodes, dtype=np.float32)
    harmonic_centrality = np.zeros(n_nodes, dtype=np.float32)

    # Memory cleanup
    del G_rev, edges
    gc.collect()

    # Main loop
    print("Starting CPU loop...")
    t = 0
    changed = True

    while changed:
        # Time
        start_time = time.time()

        t += 1
        changed = False

        m_src = M_A if t % 2 == 1 else M_B
        m_target = M_B if t % 2 == 1 else M_A

        # COUNTERS PROPAGATION
        m_target[:] = m_src[:]
        if len(sources) > 0:
            np.maximum.at(m_target, targets, m_src[sources])
        np.copyto(M_float, m_target, casting='safe')

        # Harmonic mean estimation
        np.multiply(M_float, -1.0, out=M_float)
        np.power(2.0, M_float, out=M_float)
        sum_regs = np.sum(M_float, axis=1)
        estimate_raw = factor / sum_regs

        # Estimate correction with linear counting
        estimate = estimate_raw.copy()
        num_zeros = np.sum(m_target == 0, axis=1) # number of 0 valued registers fro each node
        threshold = 2.5 * m
        mask = (estimate_raw <= threshold) & (num_zeros > 0) # Mask to identify nodes requiring correction

        if np.any(mask):
            # Formula: m * log(m / V)
            V = num_zeros[mask].astype(np.float32)
            estimate[mask] = m * np.log(m / V)

        # CHECK CHANGES WRT PREVIOUS ESTIMATE
        diff = estimate - prev_cardinality
        mask_changed = diff > 0.001

        if np.any(mask_changed):
            changed = True
            harmonic_centrality[mask_changed] += diff[mask_changed] * (1.0 / t)
            prev_cardinality[mask_changed] = estimate[mask_changed]

        # Time
        elapsed = time.time() - start_time

        # Count active nodes
        active_nodes = int(np.sum(mask_changed))
        print(f"End of t={t} iteration. CPU Time: {elapsed:.4f}s. Active nodes: {active_nodes}")

        if t > 1000:
            break

    # Results
    print("Returning data as pandas DataFrame...")
    return pd.DataFrame({
        'ASIN': nodes,
        'HarmonicCentrality': harmonic_centrality
    })


In [5]:
df_harmonic_scores = harmonic_CPU(G)
df_harmonic_scores.to_csv("../data/processed/harm_scores_cpu.csv", index=False)
display(df_harmonic_scores.head(5))
print(f"Computed hc for {len(df_harmonic_scores)} nodes.")

CPU setup
Computing initial hashes on CPU...
Starting CPU loop...
End of t=1 iteration. CPU Time: 15.4433s. Active nodes: 212737
End of t=2 iteration. CPU Time: 14.0596s. Active nodes: 177200
End of t=3 iteration. CPU Time: 13.5436s. Active nodes: 151752
End of t=4 iteration. CPU Time: 13.4854s. Active nodes: 135309
End of t=5 iteration. CPU Time: 13.6312s. Active nodes: 123841
End of t=6 iteration. CPU Time: 13.4969s. Active nodes: 115746
End of t=7 iteration. CPU Time: 13.6725s. Active nodes: 109974
End of t=8 iteration. CPU Time: 13.6706s. Active nodes: 105901
End of t=9 iteration. CPU Time: 13.5383s. Active nodes: 102913
End of t=10 iteration. CPU Time: 13.5716s. Active nodes: 100695
End of t=11 iteration. CPU Time: 13.4532s. Active nodes: 98941
End of t=12 iteration. CPU Time: 13.5057s. Active nodes: 97592
End of t=13 iteration. CPU Time: 13.5413s. Active nodes: 96577
End of t=14 iteration. CPU Time: 13.4856s. Active nodes: 95811
End of t=15 iteration. CPU Time: 13.5790s. Active n

,ASIN,HarmonicCentrality
0,0827229534,10890.977539
1,0738700797,9056.976562
2,0842328327,2.840921
3,1577943082,5.111011
4,0486220125,0.000000


Computed hc for 334843 nodes.


---

### GPU accelerated computation (cuPy matrices and buffer pre-allocation)

In [2]:
import numpy as np
import cupy as cp
import pandas as pd
import time
import hashlib
import gc

def harmonic_GPU(G, p=10):

    # Graph reverse
    G_rev = G.reverse()

    # Graph representation through nodes' indices
    nodes = list(G_rev.nodes())
    n_nodes = len(nodes)
    node_to_idx = {node: i for i, node in enumerate(nodes)} # mapping node -> node_index
    edges = np.array([(node_to_idx[u], node_to_idx[v]) for u, v in G_rev.edges()], dtype=np.int32)


    # Counters matrix
    m = 1 << p  # registers per counter, m = 2^p

    M_cpu = np.zeros((n_nodes, m), dtype=np.int32) # int32, as uint8 is not supported by cupy
    """
    [n_nodes x m] counter matrix; each register stores the number of trailing zeros of a 128-bit hash, max stored value < 128.
    """

    # Hash computation
    print("Computing hashes on CPU...")
    for i, node in enumerate(nodes):
        # Node hash
        h = int(hashlib.md5(str(node).encode('utf8')).hexdigest(), 16)

        # Bitwise AND to get the last p bits of h, then right shift to remove them
        j = h & (m - 1)
        w = h >> p

        # Count trailing zeroes on the remaining hash (+1)
        rho = 1
        while (w & 1) == 0 and rho < (128 - p):
            w >>= 1
            rho += 1
        M_cpu[i, j] = rho

    # GPU memory allocation and data transfer
    print("Allocating GPU memory and transferring data...")
    M_A = cp.asarray(M_cpu)
    M_B = M_A.copy()
    M_float = cp.empty_like(M_A, dtype=cp.float32)
    prev_cardinality_gpu = cp.ones(n_nodes, dtype=cp.float32)
    harmonic_centrality_gpu = cp.zeros(n_nodes, dtype=cp.float32)

    if len(edges) > 0:
        sources_gpu = cp.asarray(edges[:, 1])
        targets_gpu = cp.asarray(edges[:, 0])
    else:
        sources_gpu = cp.array([], dtype=cp.int32)
        targets_gpu = cp.array([], dtype=cp.int32)

    # HyperBall parameters
    alpha_m = 0.7213 / (1 + 1.079 / m)
    factor = alpha_m * (m ** 2)

    # Memory cleanup
    del M_cpu, edges
    gc.collect()


    # Main loop
    t = 0
    changed = True
    while changed:
        t += 1
        changed = False

        # Time
        cp.cuda.Stream.null.synchronize()
        start_time = time.time()

        m_src = M_A if t % 2 == 1 else M_B
        m_target = M_B if t % 2 == 1 else M_A

        # COUNTERS PROPAGATION
        m_target[:] = m_src[:]
        if len(sources_gpu) > 0:
            # Update target nodes with the maximum
            cp.maximum.at(m_target, targets_gpu, m_src[sources_gpu])
        M_float[:] = m_target.astype(cp.float32)

        # Harmonic mean estimation
        cp.multiply(M_float, -1.0, out=M_float)
        cp.exp2(M_float, out=M_float)
        sum_regs = cp.sum(M_float, axis=1)
        estimate_raw = factor / sum_regs

        # Estimate correction with linear counting
        estimate = estimate_raw.copy()
        num_zeros = cp.sum(m_target == 0, axis=1) # number of 0 valued registers fro each node
        threshold = 2.5 * m
        mask = (estimate_raw <= threshold) & (num_zeros > 0) # Mask to identify nodes requiring correction

        if cp.any(mask):
            # Formula: m * log(m / V)
            V = num_zeros[mask].astype(cp.float32)
            estimate[mask] = m * cp.log(m / V)

        # CHECK CHANGES WRT PREVIOUS ESTIMATE
        diff = estimate - prev_cardinality_gpu
        mask_changed = diff > 0.001

        if cp.any(mask_changed):
            changed = True
            harmonic_centrality_gpu[mask_changed] += diff[mask_changed] * (1.0 / t)
            prev_cardinality_gpu[mask_changed] = estimate[mask_changed]

        # Time
        cp.cuda.Stream.null.synchronize()
        elapsed = time.time() - start_time

        # Count active nodes
        active_nodes = int(cp.sum(mask_changed))
        print(f"End of t={t} iteration. GPU Time: {elapsed:.4f}s. Active nodes: {active_nodes}")

        if t > 1000:
            break

    # Results
    print("Returning data as pandas DataFrame...")
    return pd.DataFrame({
        'ASIN': nodes,
        'HarmonicCentrality': harmonic_centrality_gpu.get()
    })

In [3]:
df_harmonic_scores = harmonic_GPU(G)
df_harmonic_scores.to_csv("../data/processed/harm_scores_gpu.csv", index=False)
display(df_harmonic_scores.head(5))
print(f"Computed hc for {len(df_harmonic_scores)} nodes.")

Computing hashes on CPU...
Allocating GPU memory and transferring data...
End of t=1 iteration. GPU Time: 2.4438s. Active nodes: 212737
End of t=2 iteration. GPU Time: 3.6608s. Active nodes: 177200
End of t=3 iteration. GPU Time: 0.8916s. Active nodes: 151752
End of t=4 iteration. GPU Time: 0.8936s. Active nodes: 135309
End of t=5 iteration. GPU Time: 0.8903s. Active nodes: 123841
End of t=6 iteration. GPU Time: 0.8916s. Active nodes: 115746
End of t=7 iteration. GPU Time: 0.8940s. Active nodes: 109974
End of t=8 iteration. GPU Time: 0.8956s. Active nodes: 105901
End of t=9 iteration. GPU Time: 0.8935s. Active nodes: 102913
End of t=10 iteration. GPU Time: 0.8948s. Active nodes: 100695
End of t=11 iteration. GPU Time: 0.8940s. Active nodes: 98941
End of t=12 iteration. GPU Time: 0.8931s. Active nodes: 97592
End of t=13 iteration. GPU Time: 0.8918s. Active nodes: 96577
End of t=14 iteration. GPU Time: 0.8980s. Active nodes: 95811
End of t=15 iteration. GPU Time: 0.8932s. Active nodes: 9

,ASIN,HarmonicCentrality
0,0827229534,10890.977539
1,0738700797,9056.976562
2,0842328327,2.840921
3,1577943082,5.111011
4,0486220125,0.000000


Computed hc for 334843 nodes.
